In [2]:
from code_tokenizer import *

code = """
my_list = [1, 2, 3, 4, 5]
min = my_list[0]
for(i = 1; i < my_list.length; i++) {
    if(my_list[i] < min) {
        min = my_list[i]
    }
}
console.log(min)
"""

print(color_by_char_type(code))



def math_layer(code):
    code = parase_groups(code)
    priority = {
        '+': 1,
        '-': 1,
        '*': 2,
        '/': 2,
    }
    operations = []
    variables = []
    result = []
    # create a binary tree from the operations using the priority
    for token, value in code:
        if(token == 'symbol'):
            if(value in priority.keys()):
                current_priority = priority[value]
            else:
                current_priority = 0
            while(len(operations) > 0 and current_priority <= operations[-1][0]):
                if(len(variables) > 0 and len(result) == 0):
                    result.append(variables.pop())
                if(len(variables) > 0):
                    result.append(variables.pop())
                if(len(operations) > 0):
                    result.append(operations.pop()[1])
            operations.append([current_priority,value])
        elif(token in ['word','number']):
            variables.append(value)
    busy = True
    while(busy):
        busy = False
        if(len(variables) > 0 and len(result) == 0):
            result.append(variables.pop())
            busy = True
        if(len(variables) > 0):
            result.append(variables.pop())
            busy = True
        if(len(operations) > 0):
            result.append(operations.pop()[1])
            busy = True
    return result

# def split(code):
#     code = parase_groups(code)
#     result = [[]]
#     cursor = [code]
#     index_stack = [0]

#     while(1):
#         if(index_stack[-1] > len(cursor[-1])):
#             index_stack.pop()
#             cursor.pop()
        

#     return code

def parase_groups(code):
    code = group(code)
    result = [[]]
    cursor = [code]
    index_stack = [0]
    context = []
    current = result[-1]

    while(1):
        if(index_stack[-1] >= len(cursor[-1])):
            index_stack.pop()
            cursor.pop()
            current = result[-1]
            if(len(index_stack) == 0):
                break
            index_stack[-1] += 1
            continue
        type, value = cursor[-1][index_stack[-1]]
        if(type in ('word','number','symbol')):
            current.append([type,value])
        if(type in ('{','[','(', '"', '\'')):
            # result = ['function'] + result 
            context.append(type)
            index_stack.append(0)
            cursor.append(value)
            if(type == '['):
                if(len(current) == 0 or current[-1][0] != 'word'):
                    current = ['array_define']
                else:
                    current = ['array_call']
            elif(type == '{'):
                if(len(current) == 0):
                    current = ['code_block']
                elif(current[-1][0] == 'word' or current[0] == 'attached_group'):
                    current = ['attached_block']
                else:
                    current = ['dictionary_define']
            elif(type == '('):
                if(len(current) == 0):
                    current = ['numeric_group']
                elif(current[-1][0] == 'word'):
                    current = ['attached_group']
                else:
                    current = ['numeric_group']
            result[-1].append(current)
            continue
        if(type == 'whitespace'):
            if('\n' in value):
                result.append([])
                current = result[-1]
        index_stack[-1] += 1

    return [item for item in result if item != []]

for i in parase_groups(code):
    print(i)


my_list = [1, 2, 3, 4, 5]
min = my_list[0]
for(i = 1; i < my_list.length; i++) {
    if(my_list[i] < min) {
        min = my_list[i]
    }
}
console.log(min)

[['word', 'my_list'], ['symbol', '='], ['array_define', ['number', '1'], ['symbol', ','], ['number', '2'], ['symbol', ','], ['number', '3'], ['symbol', ','], ['number', '4'], ['symbol', ','], ['number', '5']]]
[['word', 'min'], ['symbol', '='], ['word', 'my_list'], ['array_call', ['number', '0']]]
[['word', 'for'], ['attached_group', ['word', 'i'], ['symbol', '='], ['number', '1'], ['symbol', ';'], ['word', 'i'], ['symbol', '<'], ['word', 'my_list'], ['symbol', '.'], ['word', 'length'], ['symbol', ';'], ['word', 'i'], ['symbol', '++']], ['dictionary_define']]
[['word', 'if'], ['attached_group', ['word', 'my_list']], ['array_call', ['word', 'i']], ['symbol', '<'], ['word', 'min'], ['attached_block']]
[['word', 'min'], ['symbol', '='], ['word', 'my_list'], ['array_call', ['word', 'i']]]
[['word', 'console'], ['symbol', '.'], ['wor